In [ ]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model('bees-wasps.h5')

converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees-wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

What's the size of the converted model?

In [ ]:
# get the size of the model
import os
os.path.getsize('bees-wasps.tflite') / 1024 / 1024

To be able to use this model, we need to know the index of the input and the index of the output.
What's the output index for this model?

In [ ]:
import tensorflow.lite as tflite
interpreter = tflite.Interpreter(model_path='bees-wasps.tflite')
interpreter.allocate_tensors()

output_index = interpreter.get_output_details()[0]['index']
output_index

In [ ]:
from io import BytesIO
from urllib import request

from PIL import Image

def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.NEAREST)
    return img


In [ ]:
url = 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
image = download_image(url)
image = prepare_image(image, target_size=(150, 150))

In [ ]:
import numpy as np
def preprocess_input(x):
    # rescale by 1/255
    return x / 255.0

x = np.array(image, dtype='float32')
X = np.array([x])

X = preprocess_input(X)
X[0][0][0]

In [ ]:
input_index = interpreter.get_input_details()[0]['index']
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

In [ ]:
!docker pull agrigorev/zoomcamp-bees-wasps:v2
!docker images agrigorev/zoomcamp-bees-wasps:v2

In [ ]:
import requests

url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://habrastorage.org/webt/rt/d9/dh/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

result = requests.post(url, json=data).json()
print(result)